In [1]:
import pandas as pd
import re
import underthesea
import os

In [2]:
data_path = "./Dataset_For_Work"

In [3]:
def load_data(file_path, label):
    """
    Đọc dữ liệu từ file và gắn nhãn tương ứng.
    Args:
        file_path (str): Đường dẫn đến tệp văn bản.
        label (str): Nhãn cảm xúc (positive, neutral, negative).
    Returns:
        data (list): Danh sách chứa các câu và nhãn.
    """
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            sentence = line.strip()  # Xóa khoảng trắng ở đầu và cuối câu
            if sentence:  # Chỉ thêm những câu không rỗng
                data.append((sentence, label))
    return data

In [4]:
def clean_text(text):
    """
    Làm sạch văn bản bằng cách loại bỏ ký tự đặc biệt, số và khoảng trắng thừa.
    Args:
        text (str): Câu văn bản cần làm sạch.
    Returns:
        str: Văn bản sau khi làm sạch.
    """
    text = text.lower()  # Chuyển thành chữ thường
    text = re.sub(r'\d+', '', text)  # Loại bỏ số
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa
    return text

In [5]:
# Đọc dữ liệu từ các tệp và gắn nhãn
positive_data = load_data(os.path.join(data_path, "SA-training_positive.txt"), "positive")
neutral_data = load_data(os.path.join(data_path, "SA-training_neutral.txt"), "neutral")
negative_data = load_data(os.path.join(data_path, "SA-training_negative.txt"), "negative")

# Kết hợp tất cả dữ liệu
all_data = positive_data + neutral_data + negative_data

# Tiền xử lý văn bản
cleaned_data = [(clean_text(sentence), label) for sentence, label in all_data]

# Chuyển dữ liệu sang DataFrame
df = pd.DataFrame(cleaned_data, columns=["Sentence", "Label"])

# Hiển thị một vài dòng đầu tiên
print(df.head())

                                            Sentence     Label
0  đang xài mx dùng bình thường ngon pin trâu mỗi...  positive
1  qủa pin ngon sạc lại được bền riêng em dùng pi...  positive
2  cũng đang xài con logitech bluetooth tầm thấp ...  positive
3  logitech pin trâu thôi rôi mua con b cui ma cu...  positive
4  em có con chuột không dây k cũng đầy đủ nút bấ...  positive


In [6]:
from underthesea import word_tokenize

# Tải file stopwords
stop_words_path = "./Dataset_For_Work/vietnamese-stopwords.txt"

with open(stop_words_path, 'r', encoding='utf-8') as f:
    stop_words = set(f.read().splitlines())

# Loại bỏ stopwords
def remove_stopwords(sentence):
    # Tokenize and filter stopwords
    word_tokens = word_tokenize(sentence)
    filtered_words = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Thay mác label
def replace_labels(df, label_column):
    sentiment_map = {'positive': 1, 'negative': -1, 'neutral': 0}
    if label_column in df.columns:
        df[label_column] = df[label_column].map(sentiment_map)

df['Sentence'] = df['Sentence'].apply(remove_stopwords)
replace_labels(df, 'label')

# Cập nhật dữ liệu
print("Dữ liệu sau khi xử lý:")
print(df.head())

Dữ liệu sau khi xử lý:
                                            Sentence     Label
0  xài mx bình thường ngon pin trâu tội thằng chỗ...  positive
1               pin ngon sạc bền pin aa aaa thay thế  positive
2      xài logitech bluetooth tầm m xài đc ngon chán  positive
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...  positive
4                chuột dây k đầy đủ nút bấm pin thay  positive


In [9]:
# Thực hiện word tokenization
def tokenize_text(sentence):
    """
    Tách từ cho câu đã được tiền xử lý.
    Args:
        sentence (str): Chuỗi văn bản cần tách từ.
    Returns:
        str: Văn bản sau khi tách từ.
    """
    return ' '.join(word_tokenize(sentence))  # Tách từ và kết hợp lại thành chuỗi

# Thực hiện tách từ cho từng câu trong cột "Sentence"
df['Sentence'] = df['Sentence'].apply(tokenize_text)

# Hiển thị dữ liệu sau khi tách từ
print("Dữ liệu sau khi tách từ:")
print(df.head(10))

Dữ liệu sau khi tách từ:
                                            Sentence     Label
0  xài mx bình thường ngon pin trâu tội thằng chỗ...  positive
1               pin ngon sạc bền pin aa aaa thay thế  positive
2      xài logitech bluetooth tầm m xài đc ngon chán  positive
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...  positive
4                chuột dây k đầy đủ nút bấm pin thay  positive
5  v bluetooth trâu kém cạnh cạnh bấm chuột logit...  positive
6                                            sài cực  positive
7                              lắm ko mx anywhere ổn  positive
8  dragon war g thor xài ngon build cao cấp lắm đ...  positive
9  người thân chuột lazer logitech xài rùi hư ko ...  positive


In [11]:
from gensim.models import Word2Vec
from underthesea import word_tokenize
import pandas as pd

In [12]:
# Chuẩn bị dữ liệu cho Word2Vec (danh sách các câu dưới dạng tokenized)
tokenized_sentences = [sentence.split() for sentence in df['Sentence']]

In [13]:
# Huấn luyện mô hình Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=100,  # Kích thước vector (có thể tùy chỉnh)
                          window=5,         # Kích thước cửa sổ ngữ cảnh
                          min_count=2,      # Chỉ học từ xuất hiện >= 2 lần
                          workers=4,        # Số luồng CPU sử dụng
                          sg=1)             # Sử dụng Skip-Gram (sg=1) hoặc CBOW (sg=0)

# Lưu mô hình
word2vec_model.save("word2vec_vi.model")

In [15]:
#Lấy và kiểm tra vector của một từ cụ thể
word = "tốt"
if word in word2vec_model.wv:
    print(f"Vector của từ '{word}':\n{word2vec_model.wv[word]}")
else:
    print(f"Từ '{word}' không có trong vocabulary của Word2Vec.")

Vector của từ 'tốt':
[ 0.12427032  0.18463244  0.00591576 -0.03175231  0.06260195 -0.29050094
  0.10148292  0.20663545 -0.203469   -0.09238093 -0.16199635 -0.27261478
 -0.04451975  0.07539539  0.06600002 -0.18440935  0.02166819 -0.2391626
 -0.04690346 -0.45241073 -0.01386736  0.12362839  0.09298798 -0.01109914
 -0.01781242 -0.03849871 -0.06729291 -0.18004836 -0.24325168  0.18233955
  0.22479846  0.02558722  0.00981476 -0.16159469 -0.17927863  0.27966365
  0.02457116 -0.09304912 -0.14254352 -0.32051852  0.03765506 -0.17504385
 -0.08440095  0.12463801  0.1432225  -0.19954176 -0.19653733 -0.08124914
  0.10761157  0.05559778  0.10976934 -0.22835428 -0.03860527 -0.06904443
 -0.1096584   0.16981073  0.06517973 -0.07310096 -0.20057133 -0.09730838
  0.18031888 -0.11743724 -0.02313122  0.06410506 -0.15333247  0.09998307
  0.16109139  0.14596125 -0.32152182  0.1812165  -0.05118677  0.10137749
  0.1414953  -0.05843396  0.19180338  0.22146414  0.05007388  0.02508421
 -0.14634538  0.07982272 -0.020

In [20]:
# Biểu diễn câu bằng Word2Vec
import numpy as np

def sentence_vector(sentence, model):
    """
    Tính trung bình vector của các từ trong câu.
    Args:
        sentence (str): Câu đã tách từ.
        model: Mô hình Word2Vec đã huấn luyện.
    Returns:
        np.array: Vector biểu diễn của câu.
    """
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # Trả về vector 0 nếu không có từ nào hợp lệ
    return np.mean(word_vectors, axis=0)

# Thêm cột vector cho từng câu
df['Sentence_Vector'] = df['Sentence'].apply(lambda x: sentence_vector(x, word2vec_model))

# Hiển thị kết quả
print("Biểu diễn vector của các câu:")
print(df.head(10))


Biểu diễn vector của các câu:
                                            Sentence     Label  \
0  xài mx bình thường ngon pin trâu tội thằng chỗ...  positive   
1               pin ngon sạc bền pin aa aaa thay thế  positive   
2      xài logitech bluetooth tầm m xài đc ngon chán  positive   
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...  positive   
4                chuột dây k đầy đủ nút bấm pin thay  positive   
5  v bluetooth trâu kém cạnh cạnh bấm chuột logit...  positive   
6                                            sài cực  positive   
7                              lắm ko mx anywhere ổn  positive   
8  dragon war g thor xài ngon build cao cấp lắm đ...  positive   
9  người thân chuột lazer logitech xài rùi hư ko ...  positive   

                                     Sentence_Vector  
0  [0.06002847, 0.1967728, 0.059164625, 0.0145045...  
1  [0.11808266, 0.24715789, 0.020011934, 0.047636...  
2  [0.04018867, 0.3003286, 0.15208347, 0.06338551...  
3  [-0.054300394, 0.143

In [21]:
# Kiểm tra sự tương đồng giữa các từ
# Tìm các từ tương tự với từ 'tốt'
similar_words = word2vec_model.wv.most_similar("tốt", topn=5)
print("Các từ tương tự với 'tốt':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

Các từ tương tự với 'tốt':
lẫn: 0.9840
nhược: 0.9837
dạng: 0.9808
polar: 0.9807
lởm: 0.9803
